In [37]:
import os
import pandas as pd
import numpy as np  
import cv2
from sklearn.model_selection import StratifiedShuffleSplit
import os 
import pickle
import tensorflow as tf
import numpy as np
import time
from tensorflow import keras
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image

In [38]:
root_ = "D:/desktop/Code/Python/UndergradProject/EricData/Dataset"
CATEGORIES= ['with_mask', 'without_mask']
images = []
images_with_labels=[]

for category in CATEGORIES:
    folder_path = os.path.join(root_, category)
    img_names = os.listdir(folder_path)
    
    for img_name in img_names:
        im_path = os.path.join(folder_path,img_name)
        img = cv2.imread(im_path)
        
        resize = cv2.resize(img,(300,300))
        images.append(resize)
        
        fullstring = folder_path
        substring = 'without_mask'
        
        if substring in fullstring:
            images_with_labels.append(0)
        else:
            images_with_labels.append(1)
            
images = np.array(images)

In [39]:
y = np.reshape(images_with_labels, (len(images_with_labels), 1))

In [40]:
split = StratifiedShuffleSplit(test_size=0.2, random_state=42)

for train_index, test_index in split.split(images, images_with_labels):
    X_train, y_train = images[train_index], y[train_index]
    X_test, y_test = images[test_index], y[test_index]

In [42]:
X_train.shape, X_test.shape

((8833, 300, 300, 3), (2209, 300, 300, 3))

In [43]:
y_train.shape, y_test.shape

((8833, 1), (2209, 1))

In [44]:
labels = y_train.reshape(-1)

In [45]:
split = StratifiedShuffleSplit(test_size=0.1, random_state=42)

for train_index, test_index in split.split(X_train,list(labels)):
    X_train_, y_train_ = X_train[train_index], y_train[train_index]
    X_val, y_val = X_train[test_index], y_train[test_index]

In [46]:
X_train_.shape, y_train_.shape

((7949, 300, 300, 3), (7949, 1))

In [47]:
X_val.shape, y_val.shape

((884, 300, 300, 3), (884, 1))

In [48]:
pre_trained_model = InceptionV3(input_shape=(300, 300, 3), include_top=False, weights='imagenet')

In [50]:
pre_trained_model.summary()

][0]             
__________________________________________________________________________________________________
conv2d_351 (Conv2D)             (None, 17, 17, 192)  147456      average_pooling2d_33[0][0]       
__________________________________________________________________________________________________
batch_normalization_342 (BatchN (None, 17, 17, 192)  576         conv2d_342[0][0]                 
__________________________________________________________________________________________________
batch_normalization_345 (BatchN (None, 17, 17, 192)  576         conv2d_345[0][0]                 
__________________________________________________________________________________________________
batch_normalization_350 (BatchN (None, 17, 17, 192)  576         conv2d_350[0][0]                 
__________________________________________________________________________________________________
batch_normalization_351 (BatchN (None, 17, 17, 192)  576         conv2d_351[0][0]          

In [51]:
base_input = pre_trained_model.layers[0].input
base_output = layers.Flatten()(pre_trained_model.layers[-12].output)

In [52]:

final_output = layers.Dense(1, activation='sigmoid')(base_output)

In [53]:
model = keras.Model(inputs=base_input, outputs=final_output)

In [54]:
model.compile(optimizer = "Adam", 
              loss = 'binary_crossentropy', 
              metrics = ['acc'])

In [55]:
model.fit(
    X_train_,
    y_train_,
    batch_size=32,
    epochs = 3,
    validation_data=(X_val, y_val))

Epoch 1/3
249/249 [==============================] - 115s 401ms/step - loss: 0.1962 - acc: 0.9439 - val_loss: 175.7260 - val_acc: 0.5577
Epoch 2/3
249/249 [==============================] - 95s 381ms/step - loss: 0.0976 - acc: 0.9734 - val_loss: 0.1516 - val_acc: 0.9649
Epoch 3/3
249/249 [==============================] - 95s 381ms/step - loss: 0.0415 - acc: 0.9867 - val_loss: 0.1113 - val_acc: 0.9785


In [56]:
model.evaluate(X_test, y_test)

70/70 [==============================] - 7s 103ms/step - loss: 0.0852 - acc: 0.9846


[0.08515658229589462, 0.9846084117889404]

In [59]:
X_test[0].shape

(300, 300, 3)

In [81]:
model.predict(np.expand_dims(resize, axis=0))

array([[0.9980241]], dtype=float32)

In [62]:
y_test[0]

array([1])

In [64]:
y_test[2]

array([0])

In [80]:
 TestImg = "D:/desktop/12.jpg"
 img = cv2.imread(TestImg)
 resize = cv2.resize(img,(300,300))
 print(resize.shape)

(300, 300, 3)
